## Importing libraries

In [ ]:
from collections import Counter

## Model initialization

Getting the model weights from Google Drive

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

Starting the session and loading the model

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Obtaining the results

Splitting the validation set into question and answers sets

In [17]:
val_questions = []
val_answers = []

for v in val:
  val_questions.append(v.split('\n')[0])
  val_answers.append(v.split('\n')[1])

Generating the answers to questions from the validation set

In [ ]:
eval_answers = []
for question in val_questions:
  answer = gpt2.generate(sess,
                length=300,
                temperature=0.7,
                top_k=40,
                prefix=question,
                nsamples=1,
                batch_size=1,
                )
  eval_answers.append(answer)

Computing Precision, Recal, F1-Score

In [ ]:
num_c = []
num_p = []
num_g = []

for a in range(len(eval_answers)):

  common = collections.Counter(val_answers[a].split()) & collections.Counter(eval_answers[a].split()) # tokens shared between gold and predicted answers
  num_common = sum(common.values())

  num_pred = len(str(eval_answers[a]).split()) # the number of predicted tokens

  num_gold = len(str(val_answers[a]).split()) # the number of gold tokens

  num_c.append(num_common)
  num_p.append(num_pred)
  num_g.append(num_gold)

precision = 1.0 * sum(num_c) / sum(num_p) # the num of tokens shared between gold and predicted answers / the num of predicted tokens
recall = 1.0 * sum(num_c) / sum(num_g) # the num of tokens shared between gold and predicted answers / the num of gold tokens
f1_score = (2 * precision * recall) / (precision + recall)